In [1]:
import time
from datetime import datetime

import akshare as ak


In [2]:

"""
Version 2:

1. 以00 60开头 [主板]
2. 2.8 < 换手率 < 20, 量比大于1.2 [足够活跃, 又不至于出货]
3. 振幅 > 6.5% 
4. (-2 <= 涨跌幅 <= 5.5) [收在-2到5.5之间]
5. 上影线是实体的1.5倍以上
6. 下引线小于2.3

7. 10MA > 20MA 
8. 最高价高于50个交易日的收盘价
9. 收盘价离10MA比较近（-4%内）,并且站上10日线的
10.收盘价离20MA不超过（-8内）
"""

'\nVersion 2:\n\n1. 以00 60开头 [主板]\n2. 2.8 < 换手率 < 20, 量比大于1.2 [足够活跃, 又不至于出货]\n3. 振幅 > 6.5% \n4. (-2 <= 涨跌幅 <= 5.5) [收在-2到5.5之间]\n5. 上影线是实体的1.5倍以上\n6. 下引线小于2.3\n\n7. 10MA > 20MA \n8. 最高价高于50个交易日的收盘价\n9. 收盘价离10MA比较近（-4%内）,并且站上10日线的\n10.收盘价离20MA不超过（-8内）\n'

In [5]:

def ma_calculate(df):
    ma_list = [5, 10, 20, 60, 120]

    for ma_len in ma_list:
        df['ma_' + str(ma_len)] = df['收盘'].rolling(ma_len).mean()

    return df.fillna(0)


def max_close_price_calculate(df):
    # 计算45个交易日的最高收盘价
    df['max_45'] = df['收盘'].rolling(45).max()
    return df


def detail_stock(symbol):
    """
    '日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率'
    :return:
    """
    time.sleep(4)  # 担心封ip
    print(symbol)
    today = datetime.now().strftime('%Y%m%d')
    stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date="20220910", end_date=today,
                                            adjust="qfq")

    df = ma_calculate(stock_zh_a_hist_df)
    df = max_close_price_calculate(df)

    # 偏离10日均线点数
    df['deviate_ma_10'] = (df['收盘'] - df['ma_10']) / df['收盘'] * 100
    # 偏离20日均线点数
    df['deviate_ma_20'] = (df['收盘'] - df['ma_20']) / df['收盘'] * 100

    return df


def get_trade(symbol):
    """
    获取行业信息
    :param symbol:
    :return:
    """
    df = ak.stock_individual_info_em(symbol=symbol)
    dct = dict(zip(df['item'], df['value']))

    return dct['行业']


def filter_stock(symbol):
    """
    10MA > 20MA
    最高价高于45个交易日的收盘价
    股价运行于季线之上
    收盘价离10AM比较近（-4%内）,并且站上10日线的
    收盘价离20AM比较近（-8%内）,盈亏比考虑，生命线20日线不能太远
    :param symbol:
    :return:
    """
    time.sleep(2)
    df = detail_stock(symbol)
    data = df.iloc[-1].to_dict()

    cond1 = data['ma_10'] >= data['ma_20']*0.98
    cond2 = data['最高'] * 1.05 >= data['max_45']
    cond3 = -0.25 < data['deviate_ma_10'] <= 5
    cond4 = -0.25 < data['deviate_ma_20'] <= 8
    cond5 = data['收盘'] >= data['ma_60']

    if cond1 and cond2 and cond3 and cond4 and cond5:
        return True

    return False


In [6]:


def generate():
    stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
    print(len(stock_zh_a_spot_em_df))
    df = stock_zh_a_spot_em_df[
        stock_zh_a_spot_em_df['代码'].str.startswith('00') | stock_zh_a_spot_em_df['代码'].str.startswith('60')]

    print(len(df))
    """
    Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高', '最低',
           '今开', '昨收', '量比', '换手率', '市盈率-动态', '市净率', '总市值', '流通市值', '涨速', '5分钟涨跌',
           '60日涨跌幅', '年初至今涨跌幅'],
    """
    # TODO 最近上引线也可以，但是需要趋势向上

    # 流通市值在 15亿 到 500亿之间
    min_market_value = 18 * (10 ** 8)
    max_market_value =1000 * (10 ** 8)
    df = df[(min_market_value <= df['流通市值']) & (df['流通市值'] <= max_market_value)]

    # max price
    df['max_price'] = df[['今开', '最新价']].max(axis=1)
    df['min_price'] = df[['今开', '最新价']].min(axis=1)

    # 上引线长度
    df['upper_lead'] = (df['最高'] - df['max_price']) / df['昨收'] * 100

    # 实体长度
    df['len_of_entity'] = abs(df['最新价'] - df['今开']) / df['昨收'] * 100

    # 下引线长度
    df['lower_lead'] = (df['min_price'] - df['最低']) / df['昨收'] * 100

    # 条件筛选
    # 换手率大于 2.8, 小于25 【真实的换手率其实需要排除掉一些实控人的资金，再计算的，这里不算严格】
    df = df[(df['换手率'] > 2.8) & (df['换手率'] < 25)]

    # 量比大于1.2 放量的
    df = df[df['量比'] > 1.2]

    # 振幅大于 6.5
    df = df[df['振幅'] > 6]

    # (-2 <= 涨跌幅 <= 5.5) [收在-2到5.5之间]
    df = df[(-2 < df['涨跌幅']) & (df['涨跌幅'] <= 5.5)]

    # 上影线是实体的0.5倍以上 【放宽条件，后续进行人工筛选】
    ndf = df[df['upper_lead'] >= df['len_of_entity'] * 0.5]

    # 下引线小于2.3
    ndf = ndf[ndf['lower_lead'] < 2.5]
    
    print('需要查询的数量：', len(ndf))

    # 根据均线和趋势过滤
    ndf['filter'] = ndf['代码'].map(lambda x: filter_stock(x))

    ndf.insert(loc=3, column='行业', value=ndf['代码'].map(lambda x: get_trade(x)))

    ndf = ndf[ndf['filter']]

    print(len(ndf))
    # ndf.to_excel('result.xlsx', index=False)
    return ndf


# generate()
ndf = generate()
ndf[['代码', '名称', '行业', '最新价', '涨跌幅']]


5357
3377
需要查询的数量： 22
603388
002817
600481
600605
600071
603860
002805
002800
600556
002941
001301
002997
002173
600536
002302
002929
603603
002167
603138
600749
600728
002912
2


,代码,名称,行业,最新价,涨跌幅
246,600481,双良节能,通用设备,15.84,4.55
380,600556,天下秀,文化传媒,9.27,3.46


In [13]:
# 回测方案：
# 直接买入，

# 筛选股票的上引线
# 第二天开盘价买入
# 第三天竞价卖出

# TODO
提前缓存数据
画图
三线缠绕


# 要点
5日线-20日线 差距在2.2个点内
5日前，10日线 大于 20日线（避免下跌趋势）
似乎十字星成功率高很多

# 不预测涨跌，要做的是都有应对方案

In [ ]:
e0d8734ed0a3476fa8d630dc004ed1bd

c8e2f9ac0d044cd8917159b74573b248



python3 tools/infer/predict_rec.py --image_dir='/home/aistudio/images/imgs/' --rec_model_dir='./inference/rec_robustcanner/' --rec_algorithm='RobustScanner' --rec_image_shape='3,48,48,160' --rec_char_dict_path='chinese_cht_dict.txt'


python3 tools/export_model.py -c configs/rec/rec_r31_robustscanner.yml  -o Global.pretrained_model=./output/rec/rec_r31_robustscanner/best_accuracy Global.save_inference_dir=./inference/rec_robustcanner --fixed_input_shape=[-1,3,48,160]


In [103]:
7*6*5*4*3*2

5040

In [106]:
len('0123456789abcdefghijklmnopqrstuvwxyz')

36

72033.89830508475